### using textrank to get key sentences

In [1]:
# 출력이 너무 길어지지 않게하기 위해 찍지 않도록 했으나 
# 실제 학습 할 때는 아래 두 줄을 주석처리 하는 것을 권장한다.
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(np.__version__)

1.1.0
1.18.5


In [3]:
# pandas 설정: 모든 row와 모든 column을 출력하게 하고, 각 column 별 출력되는 글자수는 50자 이내
pd.set_option('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 50)

In [4]:
# 위의 설정으로 간단하게 보는 df_specific
df_specific = pd.read_csv("./data_wrangled/df_specific_wrangle.csv", encoding="utf-8")
df_specific.head(2)

,대학코드,대학명,gen_info,env_info,food_info,study_info,office_info,facil_info,mhct_info,help_info,etc_info
0,AR000004,Universidad del Salvador,아르헨티나의 대다수 대학교는 한국처럼 캠퍼스가 있는 것이 아니라 도시 내에 몇개의 ...,대학교가 도시 여러곳에 분포하고 있으므로 각기 다르다. 하지만 남녀를 불문하고 부에...,기숙사는 학교에서 제공하지 않고 두가지 옵션이 주어진다. 홈스테이 혹은 게스트 하우...,아르헨티나에서의 대학 수업은 주로 오전 혹은 저녁에 있다. 대부분의 학생들이 일을하...,EL SALVADOR대학교는 국제 교육부가 잘 되있는 학교이다. 아르헨티나 대학교들...,부대시설로는 헬스장이 있다고 알고 있으나 시내와 먼 곳에 있었다. 동아리는 몇가지 ...,딱히 컬쳐쇼크를 받을 만 한 요소는 없었던 것 같다. 하지만 처음에 언어적인 면에서...,"살바도르 대학교 국제처에서 비자문제나 숙소문제(홈스테이경우) 등은 해결해 주고, 특...",NaN
1,AU000019,Australian National University,"ANU has a very big campus, but unlike Yonsei, ...","The city center, called CIVIC, is 20min walk a...","There are many housing options, but I chose to...","In my personal opinions, classes are generally...","Overall, I did not have to ask for help much f...","There were many clubs for you to join at ANU, ...",I did have some culture shock during my first ...,"I was helped a lot, especially during the firs...","Overall, my stay in Australia has been nothing..."


In [5]:
# 위의 설정으로 간단하게 보는 df_specific
df_specific.tail(2)

,대학코드,대학명,gen_info,env_info,food_info,study_info,office_info,facil_info,mhct_info,help_info,etc_info
468,US000274,Whitworth College,내가 다녀온 교환대학은 Washington State 에 위치한 Whitworth ...,Whitworth College는 Seattle에서 비행기로 40분 정도 떨어져 있...,나는 Whitworth College에서 기숙사에 살았다. ISEP으로 가게 되면 ...,내가 미국에 가기 전에 가장 두려워했던 것은 바로 수업이었다. 어학연수와는 달리 교...,이 학교에서 내가 마음 편하게 지낼 수 있었던 것은 국제교육부 스텝들이 너무 따뜻하...,"기타 학교의 부대 시설로는 fitness center, music hall, soc...",NaN,NaN,처음에 Whitworth College를 ISEP으로부터 배정 받았을 때 현실적으로...
469,US000278,Wittenberg University,Wittenberg는 Ohio주의 작은 도시 Springfield에 위치해 있다. ...,"Springfield의 downtown에는 우체국, 은행(huntington) 정도...","1,2학년과 국제학생들 (1년 이하로 있는 학생들)은 모두 기숙사에서 생활하게 된다...",도서관은 Thomas library가 main이고 다른 단과대학 건물에 독서실이 있...,"국제교류부는 Hollenbeck 건물 1층에 있는데, Joann 과 Betsy 두 ...",Wittenberg의 가장 큰 장점. nWittenberg는 학비가 무척 비싼 학교...,"한국에서는 흔치 않은 gay, lesbian, transgender 등을 만날 수 ...",Wittenberg는 국제학생들에게 host family를 붙여준다. 비록 같이 살...,이처럼 wittenberg는 시골에 있는 작은 학교지만 지리적인 위치로 인한 단점보...


In [6]:
print(df_specific.columns)
print(df_specific.shape)

Index(['대학코드', '대학명', 'gen_info', 'env_info', 'food_info', 'study_info',
       'office_info', 'facil_info', 'mhct_info', 'help_info', 'etc_info'],
      dtype='object')
(470, 11)


In [7]:
# pandas 설정: 각 column 별 (혹은 한 cell에서) 출력되는 글자수 제한을 없애기
pd.set_option('display.max_colwidth', None)

In [8]:
import re
# removing other letters except for korean letters from the string data
def remove_junk(str_data):
    str_data = str_data.replace("\\\\n", "")
    str_data = str_data.replace("\\n", "")
    str_data = str_data.replace("\n", "")
    str_data = str_data.replace("\\", "")
    str_data = str_data.replace("\\t", "")
    str_data = str_data.replace("NaN", "")

    # print(str_data) makes an error. too much of data
    # 한글, english 빼고 전부 날리기
    # str_data = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z|0-9|.,]', ' ', str_data)

    # 한글 빼고 다 날리기
    str_data = re.sub('[^가-힣ㄱ-ㅎㅏ]', ' ', str_data)

    # replace multiple spaces into single space
    str_data = ' '.join(str_data.split())
    return str_data

def access_univ_info(dataframe, column, univ_code):
    df_row = dataframe.loc[dataframe["대학코드"] == univ_code]
    str_univ_info = df_row[column].to_string(index=False).lstrip()
    str_univ_info = remove_junk(str_univ_info)
    return str_univ_info


# removing junks only from the string data
def remove_junk_only(str_data):
    str_data = str_data.replace("\\\\n", "")
    str_data = str_data.replace("\\n", "")
    str_data = str_data.replace("\n", "")
    str_data = str_data.replace("\\", "")
    str_data = str_data.replace("\\t", "")
    str_data = str_data.replace("NaN", "")

    # replace multiple spaces into single space
    str_data = ' '.join(str_data.split())
    return str_data

# accessing university info of given column (= accessing one cell)
def access_univ_info_raw(dataframe, column, univ_code):
    df_row = dataframe.loc[dataframe["대학코드"] == univ_code]
    str_univ_info = df_row[column].to_string(index=False).lstrip()
    str_univ_info = remove_junk_only(str_univ_info)
    return str_univ_info

# collecting all column values into one string
def column_to_string(dataframe, column_name: str):
    str_corpus = dataframe[column_name].to_string(index=False).lstrip()
    str_corpus = remove_junk_only(str_corpus)
    return str_corpus

In [9]:
column_data = "gen_info"
univ_code = "DK000003"
str_example = access_univ_info_raw(df_specific, column_data, univ_code)
str_example[:100]

'코펜하겐 대학교는 자연과학캠퍼스, 인문캠퍼스 등 코펜하겐 시내 곳곳에 흩어져 있습니다. 저는 조경건축 수업을 듣기 위해 Frederiksberg 에 위치한 자연과학 캠퍼스도 다니고'

In [10]:
# collecting all column values into one string
corpus = column_to_string(df_specific, column_data)
print("글자수:", len(corpus))
corpus[:100]

글자수: 4648952


'아르헨티나의 대다수 대학교는 한국처럼 캠퍼스가 있는 것이 아니라 도시 내에 몇개의 건물이 대학교 건물이라서 수업에 맞는 건물로 수업을 들으러 가면 된다. 도시의 대중교통이 꽤 잘 '

In [11]:
from textrankr import TextRank

column_data = "gen_info"
univ_code = "DK000003"
str_example = access_univ_info(df_specific, column_data, univ_code)
str_example = str_example.replace("니다", "니다.")

textrank = TextRank(str_example)
print(textrank.summarize())  # gives you some text
textrank.summarize(10, verbose=False)  # up to 3 sentences, returned as list

어둡고 추운 겨울을 견디고 오는 코펜하겐의 봄 여름 날씨는 환상적입니다.
는 덴마크 수도 코펜하겐에 위치하고 있어요 캠퍼스는 시내에 개로 나누어져 있습니다.
코펜하겐은 위도가 높아서 여름에는 해가 유난히 길고 시에 밝아지기 시작해서 시 넘어 해가 짐 겨울에는 매우 짧습니다.


['코펜하겐 대학의 캠퍼스는 도시 곳곳에 흩어져 있는데 저는 봄학기에는 문과대학이 위치한 에서 가을학기에는 사회과학대학이 위치한 에서 수업을 들었습니다.',
 '그러니 가을 겨울학기에 가는 분들은 날씨가 추워지기 전에 코펜하겐을 둘러보는 것을 추천합니다.',
 '코펜하겐의 여름은 날씨가 굉장히 좋고 아름답습니다.',
 '어둡고 추운 겨울을 견디고 오는 코펜하겐의 봄 여름 날씨는 환상적입니다.',
 '코펜하겐은 여름이 짧고 월 말 정도부터 추워지기 시작합니다.',
 '는 덴마크 수도 코펜하겐에 위치하고 있어요 캠퍼스는 시내에 개로 나누어져 있습니다.',
 '코펜하겐은 위도 북쪽에 위치하여 여름에는 낮이 길고 겨울에는 밤이 긴 특징을 가지 우선 캠퍼스가 여러개로 나뉘어져 있습니다.',
 '코펜하겐은 위도가 높아서 여름에는 해가 유난히 길고 시에 밝아지기 시작해서 시 넘어 해가 짐 겨울에는 매우 짧습니다.',
 '코펜하겐 대학교는 캠퍼스가 따로 없이 코펜하겐 시내 곳곳에 학교 건물이 퍼져 있습니다.',
 '코펜하겐은 덴마크의 수도이고 서울에 비한다면 그리 크지 않은 도시입니다.']

In [12]:
from textrankr import TextRank

column_data = "gen_info"
univ_code = "DK000003"

str_example = access_univ_info_raw(df_specific, column_data, univ_code)

textrank = TextRank(str_example)
print(textrank.summarize())  # gives you some text
textrank.summarize(10, verbose=False)  # up to 3 sentences, returned as list

코펜하겐의 여름은 날씨가 굉장히 좋고 아름답습니다.
어둡고 추운 겨울을 견디고 오는 코펜하겐의 봄, 여름 날씨는 환상적입니다.
코펜하겐 대학은 캠퍼스가 아닙니다.


['코펜하겐의 겨울날씨는 체감상 한국의 겨울날씨보다 우중충하고 바람이 많고 잔비가 수시로 내려서 더 춥게 느껴졌습니다.',
 '코펜하겐의 기후는 (제가 갔던 봄, 여름 기준) 대체로 서늘한 날씨지만, 겨울~초봄에는 한국보다 기온이 높아서 생각만큼 춥지는 않습니다.',
 '그러니 가을-겨울학기에 가는 분들은 날씨가 추워지기 전에 코펜하겐을 둘러보는 것을 추천합니다.',
 '코펜하겐의 여름은 날씨가 굉장히 좋고 아름답습니다.',
 '어둡고 추운 겨울을 견디고 오는 코펜하겐의 봄, 여름 날씨는 환상적입니다.',
 '코펜하겐 대학은 여러개의 캠퍼스로 나뉘어 있습니다.',
 '앞서 많은 학생들이 기술하였듯 덴마크 코펜하겐 대학은 캠퍼스 대학이 아니라 단과대학 형식으로 도시 곳곳에 건물이 위치해 있습니다.',
 '코펜하겐 대학교는 캠퍼스가 따로 없이 코펜하겐 시내 곳곳에 학교 건물이 퍼져 있습니다.',
 '코펜하겐 대학은 캠퍼스가 아닙니다.',
 '코펜하겐은 덴마크의 수도이고, 서울에 비한다면 그리 크지 않은 도시입니다.']

In [13]:
from textrankr import TextRank

column_data = "gen_info"
univ_code = "DK000003"

str_example = access_univ_info_raw(df_specific, column_data, univ_code)

textrank = TextRank(str_example)
print(textrank.summarize())  # gives you some text
textrank.summarize(10, verbose=False)  # up to 3 sentences, returned as list

코펜하겐의 여름은 날씨가 굉장히 좋고 아름답습니다.
어둡고 추운 겨울을 견디고 오는 코펜하겐의 봄, 여름 날씨는 환상적입니다.
코펜하겐 대학은 캠퍼스가 아닙니다.


['코펜하겐의 겨울날씨는 체감상 한국의 겨울날씨보다 우중충하고 바람이 많고 잔비가 수시로 내려서 더 춥게 느껴졌습니다.',
 '코펜하겐의 기후는 (제가 갔던 봄, 여름 기준) 대체로 서늘한 날씨지만, 겨울~초봄에는 한국보다 기온이 높아서 생각만큼 춥지는 않습니다.',
 '그러니 가을-겨울학기에 가는 분들은 날씨가 추워지기 전에 코펜하겐을 둘러보는 것을 추천합니다.',
 '코펜하겐의 여름은 날씨가 굉장히 좋고 아름답습니다.',
 '어둡고 추운 겨울을 견디고 오는 코펜하겐의 봄, 여름 날씨는 환상적입니다.',
 '코펜하겐 대학은 여러개의 캠퍼스로 나뉘어 있습니다.',
 '앞서 많은 학생들이 기술하였듯 덴마크 코펜하겐 대학은 캠퍼스 대학이 아니라 단과대학 형식으로 도시 곳곳에 건물이 위치해 있습니다.',
 '코펜하겐 대학교는 캠퍼스가 따로 없이 코펜하겐 시내 곳곳에 학교 건물이 퍼져 있습니다.',
 '코펜하겐 대학은 캠퍼스가 아닙니다.',
 '코펜하겐은 덴마크의 수도이고, 서울에 비한다면 그리 크지 않은 도시입니다.']

In [14]:
from textrankr import TextRank

column_data = "gen_info"
univ_code = "JP000017"
str_example = access_univ_info_raw(df_specific, column_data, univ_code)

textrank = TextRank(str_example)
print(textrank.summarize())  # gives you some text
print(textrank.summarize(10, verbose=False))  # up to 3 sentences, returned as list

나가사키에 위치해 있고 나가사키 대학교는 3개의 캠퍼스로 나뉜다.
나가사키대학은 나가사키현의 나가사키시에 위치해 있으며 3개의 작은 캠퍼스로 이루어져 있다.
나가사키는 상당히 남쪽에 위치하기 때문에 겨울에도 그렇게 추운편이 아니다.
['나가사키는 일본 큐슈의 남쪽에 위치한 작은 항구도시로, 제2차 세계대전 당시 히로시마와 함께 원폭이 떨어진 곳으로 유명하다.', '나가사키에 위치해 있고 나가사키 대학교는 3개의 캠퍼스로 나뉜다.', '나가사키대학은 일본 나가사키현 나가사키시에 경제학부/의,약학부/그 외 학부의 3개의 캠퍼스로 이루어져있다.', '기후는 우리나라의 기후에서 여름은 더 덥고 겨울은 더 춥게 느껴졌다.', '나가사키대학교는 일본의 가장 남쪽 섬인 큐슈의 나가사키시에 있다.', 'n 나가사키 대학교는 3개의 캠퍼스가 있는데 교환학생(NISP 프로.', '나가사키는 오키나와를 제외한 일본 최남서쪽에 위치한 작은 항구도시이다.', '나가사키대학은 나가사키현의 나가사키시에 위치해 있으며 3개의 작은 캠퍼스로 이루어져 있다.', '나가사키는 상당히 남쪽에 위치하기 때문에 겨울에도 그렇게 추운편이 아니다.', '나가사키는 꽤 남쪽에 위치한다.']


In [15]:
from textrankr import TextRank

column_data = "gen_info"
univ_code = "CN000016"
str_example = access_univ_info_raw(df_specific, column_data, univ_code)

textrank = TextRank(str_example)
print(textrank.summarize())  # gives you some text
print(textrank.summarize(10, verbose=False))  # up to 3 sentences, returned as list

지리적으로는 홍콩 섬에 위치해 있기 때문에 지리적으로는 탁월합니다.
1. 홍콩대 위치/크기홍콩대학교는 홍콩섬에 위치해있습니다.
홍콩대학교는 그렇게 큰 학교는 아닙니다.
['지리적으로는 홍콩 섬에 위치해 있기 때문에 지리적으로는 탁월합니다.', '홍콩대학교 위치는 최고입니다.', '1. 홍콩대 위치/크기홍콩대학교는 홍콩섬에 위치해있습니다.', '학교 위치는 홍콩에 있는 다른 어느 대학들보다도 가장 좋은 곳에 있습니다.', '홍콩대학교는 그렇게 큰 학교는 아닙니다.', '홍콩대학교는 홍콩 섬의 북서쪽에 위치해있는데 캠퍼스 자체는 우리 학교보다는 작은 편입니다.', '홍콩대학교의 크기는 홍콩섬이 작기 때문에 그리 크지는 않지만 n아늑하고 이동하기 편리하게 되있었다.', '홍콩대학교는 홍콩의 Kowloon 섬에 위치하고 있다.', '홍콩은 홍콩섬과 구룡반도 그리고 신계지로 이루어져있는데, 홍콩대학교는 홍콩섬에 위치하고 있다.', '위치 : 홍콩대학은 홍콩섬의 북서쪽에 위치하고 있습니다.']


In [30]:
from textrankr import TextRank

def yield_summary(df_specific, column_data, univ_code):
    string_data = access_univ_info_raw(df_specific, column_data, univ_code)

    textrank = TextRank(string_data)
    # print(textrank.summarize())  # gives you some text
    summarized_list = textrank.summarize(6, verbose=False)

    summary_text_md = ""

    for sentence in summarized_list:
        summary_line = \
f"""* {sentence}
"""
        summary_text_md += summary_line

    return summary_text_md

In [31]:
column_data = "gen_info"
univ_code = "CN000016"

print(yield_summary(df_specific, column_data, univ_code))

* 지리적으로는 홍콩 섬에 위치해 있기 때문에 지리적으로는 탁월합니다.
* 홍콩대학교 위치는 최고입니다.
* 1. 홍콩대 위치/크기홍콩대학교는 홍콩섬에 위치해있습니다.
* 홍콩대학교는 그렇게 큰 학교는 아닙니다.
* 홍콩대학교의 크기는 홍콩섬이 작기 때문에 그리 크지는 않지만 n아늑하고 이동하기 편리하게 되있었다.
* 홍콩은 홍콩섬과 구룡반도 그리고 신계지로 이루어져있는데, 홍콩대학교는 홍콩섬에 위치하고 있다.

